In [1]:
import pandas as pd
import numpy as np

In [3]:
import sklearn

In [9]:
#載入資料
df = pd.read_csv("./data/train.csv", encoding='big5')

X_test = pd.read_csv("./data/test.csv", encoding='big5')

/home/fio/anaconda3/envs/gpu/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (17,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
#把有ＮＡ的都砍掉
cols_with_missing = [col for col in df.columns if df[col].isnull().any()] 

df.drop(cols_with_missing, axis=1, inplace=True)
X_test.drop(cols_with_missing, axis=1, inplace=True)

In [11]:
#提取答案
train_y = df['Y1']=='Y'
df.drop(columns="Y1", inplace=True)
X = df.copy()
train_y_all = train_y+1-1
train_y_all

0        0
1        0
2        0
3        0
4        0
        ..
99995    0
99996    1
99997    1
99998    0
99999    0
Name: Y1, Length: 100000, dtype: int64

In [12]:
#把object類型都先砍掉
s = (X.dtypes == 'object')
object_cols = list(s[s].index)

In [13]:
object_cols[0:5]

['AGE',
 'CHARGE_CITY_CD',
 'CONTACT_CITY_CD',
 'LAST_A_CCONTACT_DT',
 'LAST_A_ISSUE_DT']

In [14]:
x_train = X.drop(columns = object_cols)
x_test = X_test.drop(columns=object_cols)

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train, train_y_all, test_size=0.5)
#x_train, y_train = train_all, train_y_all

In [17]:
x_train.shape

(50000, 15)

In [18]:
import xgboost as xgb

In [19]:
model = xgb.XGBClassifier(n_estimators=1000, tree_method='gpu_hist', slient= False)

model.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
              max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
              n_jobs=1, nthread=None, objective='binary:logistic',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              seed=None, silent=True, slient=False, subsample=1,
              tree_method='gpu_hist')

In [20]:
result = model.predict(x_valid)

In [21]:
sklearn.metrics.accuracy_score(result, y_valid)

0.97876

以上程式碼顯示，使用XGboost正確率達0.97<br>下面在針對測試資料做運算

In [24]:
#載入資料
df = pd.read_csv("./data/train.csv", encoding='big5')

X_test = pd.read_csv("./data/test.csv", encoding='big5')

In [25]:
#把有ＮＡ的都砍掉
cols_with_missing = [col for col in df.columns if df[col].isnull().any()] 

df.drop(cols_with_missing, axis=1, inplace=True)
X_test.drop(cols_with_missing, axis=1, inplace=True)

In [26]:
#提取答案
train_y = df['Y1']=='Y'
df.drop(columns="Y1", inplace=True)
X = df.copy()
train_y_all = train_y+1-1

In [27]:
#把object類型都先砍掉
s = (X.dtypes == 'object')
object_cols = list(s[s].index)

In [28]:
object_cols[0:5]

['AGE',
 'CHARGE_CITY_CD',
 'CONTACT_CITY_CD',
 'LAST_A_CCONTACT_DT',
 'LAST_A_ISSUE_DT']

In [29]:
x_train = X.drop(columns = object_cols)
x_test = X_test.drop(columns=object_cols)

In [30]:
from sklearn.model_selection import train_test_split

In [32]:
#x_train, x_valid, y_train, y_valid = train_test_split(x_train, train_y_all, test_size=0.5)
x_train, y_train = x_train, train_y_all

In [33]:
x_train.shape

(100000, 15)

In [34]:
import xgboost as xgb

In [36]:
model = xgb.XGBClassifier(n_estimators=1000, tree_method='gpu_hist')

model.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
              max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
              n_jobs=1, nthread=None, objective='binary:logistic',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              seed=None, silent=True, subsample=1, tree_method='gpu_hist')

因為想先試水溫，我把0轉成1，1轉成0<br>透過1-score來取得真正成績<br>來避免我們到時候無法解釋為什麼成果這麼好的可能發生

In [54]:
result = model.predict(x_test)
#這行註解掉即可取得實際結果
result = (result==0)+1-1

In [55]:
result.shape

(150000,)

In [56]:
submit = pd.read_csv("./data/submit_test.csv")

In [57]:
submit.head()

,CUS_ID,Ypred
0,1193,0
1,1727,0
2,3293,0
3,4328,0
4,5555,0


In [58]:
submit.Ypred = result

In [60]:
submit.head()

,CUS_ID,Ypred
0,1193,1
1,1727,1
2,3293,1
3,4328,1
4,5555,1


In [61]:
submit.to_csv("./result/0911_xgboost_reverse.csv")

成果 0.49